# TDEM Data collection
Objectives:
1. Learn to collect TDEM data.
2. Take detailed notes!
3. Plot collected $B$-field decay curves.

## Instrumentation

Use three different TDEM instruments:

For electrical soundings we will use:
* Geonics ProTEM receiver and EM-47 Transmitter
* WalkTEM with and without rigid receiver

For UXO identification, we will use:
* Geonics EM-63

### ProTEM & EM47
![ProTEM and EM47](media/protem-em47.png)

[ProTEM User Manual (on Canvas)](https://elearning.mines.edu/files/7434917)

### WalkTEM

![ABEM WalkTEM](media/WalkTEM.png "ABEM WalkTEM")

[WalkTEM User Manual (On Canvas)](https://elearning.mines.edu/files/7434925/download?download_frd=1)

### EM63

![EM61](media/em-63.png "Geonics EM61")
(Petterman 2011)

## Survey Setup

In Kafadar Commons

### Sounding experiment:
* Lay out a 40m by 40m **square** loop of wire for the Tx
    * Use wooden posts at the corners for stability.
* Flag the center

### UXO experiment:
* Determine a line spacing for the UXO grid
* Setup and place marker flags to help guide the data collection.

## Data Collection

I expect you to collect the data using each of the instruments.


### ProTEM
* Collect central loop sounding data in 20 gate mode at repetition rates of:
    * 285 Hz
    * 30 Hz
* Determine appropriate gain levels for each repetition rate.
* Collect 3 component reciever data.


### WalkTem
* Collect central loop sounding data using:
    * 